# SIRE 
### stochastic impulse response emulator

import heaps of python

In [ ]:
from __future__ import print_function
#from ipywidgets import interact, interactive, fixed, interact_manual
import os
import ipywidgets as ipyw
import matplotlib as mpl
import matplotlib.pyplot as plt
import ipyleaflet as ipyl
from datetime import datetime
import IPython
import warnings
warnings.filterwarnings("ignore")
import json
from json import dumps
import numpy as np
import pandas as pd
import pyemu

import hauraki_sire

# Land Use Sector Map

In [ ]:
with open(os.path.join("sire","sire_sector.json"),"r") as f:
    lu_data = json.load(f)
labellu = ipyw.Label(layout=ipyw.Layout(width='100%'))
def hover_handler(event=None, id=None, properties=None):
    labellu.value = properties["secConcord"]

geo = ipyl.GeoJSON(data=lu_data)
geo.on_hover(hover_handler)
mlu = ipyl.Map(center=[-37.65,175.65], zoom=10,layout=ipyw.Layout(height="10in",width="6in"))
mlu.add_layer(geo);
# ipyw.VBox([labellu,mlu])

### A bunch of widget hackery

In [ ]:
m_load = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))
m_sfr = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))
m_ucn = ipyl.Map(center=[-37.65,175.65], zoom=9,layout=ipyw.Layout(height="6in",width="3in"))


with open(os.path.join("sire","sire_sfr.json"),"r") as f:
    sfr_data = json.load(f)

with open(os.path.join("sire","sire_grid.json"),"r") as f:
    ucn_data = json.load(f)    
    
with open(os.path.join("sire","sire_grid.json"),"r") as f:
    load_data = json.load(f) 
# widget hackery
lu_mn,lu_mx = -10,10 #min and max landuse change
risk_str = ipyw.Text()
dairy_str = ipyw.Text()
hort_str = ipyw.Text()
snb_str = ipyw.Text()
forestry_str = ipyw.Text()
other_str = ipyw.Text()
str_dict = {"dairy":dairy_str,
           "hort":hort_str,"snb":snb_str,
           "forestry":forestry_str,
           "other":other_str}
def set_values(**kwargs):
    for k,v in kwargs.items():
        str_dict[k].value = str(v)
def set_risk(risk):
    risk_str.value = str(risk)
w = ipyw.interactive(set_values,dairy=(lu_mn,lu_mx,1),
                hort=(lu_mn,lu_mx,1),snb=(lu_mn,lu_mx,1),
                forestry=(lu_mn,lu_mx,1),
                other=(lu_mn,lu_mx,1))

i = IPython.display.Audio("dang.mp3",autoplay=True)


label_load = ipyw.Label(layout=ipyw.Layout(width='100%'))
def load_hover_handler(event=None, id=None, properties=None):
    label_load.value = "row:{0}, col:{1}, percent_change:{2:8.2G}".format(properties["row"],
                                                                           properties["column"],
                                                                           properties["response"])


label_sfr = ipyw.Label(layout=ipyw.Layout(width='100%'))
def sfr_hover_handler(event=None, id=None, properties=None):
    label_sfr.value = "reachid:{0}, response (mg/l):{1:8.2G}".format(properties["reachID"],properties["response"])
label_ucn = ipyw.Label(layout=ipyw.Layout(width='100%'))
def ucn_hover_handler(event=None, id=None, properties=None):
    label_ucn.value = "row:{0}, col:{1}, response (mg/l):{2:8.2G}".format(properties["row"],
                                                                           properties["column"],
                                                                           properties["response"])

def on_button_clicked(b):  
    tol = 1.0e-3
    risk = float(risk_str.value)
    lu_dict = {k:float(v.value) for k,v in str_dict.items()}
    print("calculating...",end='')
    s = datetime.now()
    loading_df,df = hauraki_sire.sire_lu_scenario(lu_dict,risk=risk)
    print("took:",(datetime.now()-s).total_seconds())
    print("plotting...",end='')
    start = datetime.now()
    #hauraki_sire.plot_sire(df,show=True)
    
    loading_df.loc[:,"rowcol"] = loading_df.apply(lambda x: "{0:03d}_{1:03d}".format(x.i+1,x.j+1),axis=1)
    loading_df.index = loading_df.rowcol
    loading_df.loc[:,"percent_change"] = (100.0 * (loading_df.loading_change/loading_df.base_load))
    loading_df.loc[:,"normed"] = mpl.colors.Normalize()(loading_df.percent_change.values)
    rdict = loading_df.percent_change.to_dict()
    colormap=mpl.cm.jet
    loading_df.loc[:,"color"] = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(loading_df.normed.values)]
    cdict = loading_df.color.to_dict()
    for i,feature in enumerate(ucn_data["features"]):
        rowcol = feature["properties"]["rowcol"]
        if pd.isnull(loading_df.loc[rowcol,"percent_change"]):
            rdict[rowcol] = 0.0
            color="#676565"
        elif rowcol not in rdict:
            rdict[rowcol] = 0.0
            color = "#676565"
        elif np.abs(loading_df.loc[rowcol,"percent_change"]) < tol:
            color="#676565"
        else:
            color = cdict[rowcol]  
        load_data["features"][i]["properties"]["style"]["fillColor"] = color
        load_data["features"][i]["properties"]["style"]["color"] = color
        load_data["features"][i]["properties"]["style"]["weight"] = 2.0
        load_data["features"][i]["properties"]["response"] = rdict[rowcol]
    geo = ipyl.GeoJSON(data=load_data)
    geo.on_hover(load_hover_handler)
    m_load.add_layer(geo) 
    
    #df = pd.read_csv(os.path.join("sire","sire_results.csv"))
    df.loc[:,"response"] *= 1000
    
    
    df_sfr = df.loc[df.obsnme.apply(lambda x: x.startswith('sfr')),:].copy()
    df_sfr.loc[:,"reach"] = df_sfr.obsnme.apply(lambda x: int(x.split('_')[0][4:]))
    df_sfr.index = df_sfr.reach
    df_sfr.loc[:,"normed"] = mpl.colors.Normalize()(df_sfr.response.values)
    rdict = df_sfr.response.to_dict()
    colormap=mpl.cm.jet
    df_sfr.loc[:,"color"] = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(df_sfr.normed.values)]
    cdict = df_sfr.color.to_dict()
    for i,feature in enumerate(sfr_data["features"]):
        reachid = feature["properties"]["reachID"]
        if np.abs(df_sfr.loc[reachid,"response"]) < tol:
            color="#676565"
        else:
            color = cdict[reachid]
        sfr_data["features"][i]["properties"]["style"]["fillColor"] = color
        sfr_data["features"][i]["properties"]["style"]["color"] = color
        sfr_data["features"][i]["properties"]["style"]["weight"] = 2.0
        sfr_data["features"][i]["properties"]["response"] = rdict[reachid]
    geo = ipyl.GeoJSON(data=sfr_data)
    geo.on_hover(sfr_hover_handler)
    m_sfr.add_layer(geo) 
    
    df_ucn = df.loc[df.index.map(lambda x: x.startswith("ucn")),:].copy()
    df_ucn = df_ucn.loc[df_ucn.response.apply(np.abs)<1.0e+10,:]
    df_ucn.loc[:, "i"] = df_ucn.obsnme.apply(lambda x: int(x.split('_')[2]))
    df_ucn.loc[:, "j"] = df_ucn.obsnme.apply(lambda x: int(x.split('_')[3]))
    df_ucn.loc[:,"rowcol"] = df_ucn.apply(lambda x: "{0:03d}_{1:03d}".format(x.i+1,x.j+1),axis=1)
    df_ucn.index = df_ucn.rowcol
    df_ucn.loc[:,"normed"] = mpl.colors.Normalize()(df_ucn.response.values)
    rdict = df_ucn.response.to_dict()
    colormap=mpl.cm.jet
    df_ucn.loc[:,"color"] = [mpl.colors.rgb2hex(d[0:3]) for d in colormap(df_ucn.normed.values)]
    cdict = df_ucn.color.to_dict()
    for i,feature in enumerate(ucn_data["features"]):
        rowcol = feature["properties"]["rowcol"]
        if rowcol not in rdict:
            rdict[rowcol] = 0.0
            color = "#676565"
        elif np.abs(df_ucn.loc[rowcol,"response"]) < tol:
            color="#676565"
        else:
            color = cdict[rowcol]  
        ucn_data["features"][i]["properties"]["style"]["fillColor"] = color
        ucn_data["features"][i]["properties"]["style"]["color"] = color
        ucn_data["features"][i]["properties"]["style"]["weight"] = 2.0
        ucn_data["features"][i]["properties"]["response"] = rdict[rowcol]
    geo = ipyl.GeoJSON(data=ucn_data)
    geo.on_hover(ucn_hover_handler)
    m_ucn.add_layer(geo) 
    print("took:",(datetime.now() - start).total_seconds())
    
button = ipyw.Button(description="Get Some Science")
button.on_click(on_button_clicked)
print("ready!")
#i

# Let's do this!

In [ ]:
r = ipyw.interact(set_risk,risk=(0.01,0.99,0.01))
display(w)
display(button)
#ipyw.HBox([ipyw.VBox([labellu,mlu]),ipyw.VBox([label_sfr,m_sfr])])
ipyw.HBox([ipyw.VBox([label_load,m_load]),ipyw.VBox([label_ucn,m_ucn]),ipyw.VBox([label_sfr,m_sfr])])